In [ ]:
import requests 
import json
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from IPython.display import HTML, display
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: | 

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install lxml

In [ ]:
!pip install html5lib


In [ ]:
from bs4 import BeautifulSoup

#### Weblink of toronto postal codes

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#### Scrape the website and read the postal code table

In [ ]:
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
results = soup.find('table', attrs={'class': 'wikitable sortable'})

rows = results.find_all('tr')
data = []

cols = rows[0].find_all('th')
headers = [ele.text.strip() for ele in cols]

for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

result = pd.DataFrame(data, columns=headers)
result = result[result['Borough']!="Not assigned"].reset_index(drop=True)
result.dtypes

In [ ]:
result.shape

In [ ]:

#geolocator = Nominatim(user_agent="Toronto_explorer")
#for code in result['Postcode']:
#    longlat = None
#    while (longlat is None):
#        print(code)
#        longlat = geolocator.geocode('{}, Toronto, ontario'.format(code))
#    lat.append(longlat.latitude)   
#    long.append(longlat.longitude)
#    print(lat)


#### Get the longitude and latitude from the geospatial file

In [ ]:
geo_df = pd.read_csv("Geospatial_Coordinates.csv")
longlat_dict = {}
for code, lt, ln in zip(geo_df['Postal Code'], geo_df['Latitude'], geo_df['Longitude']):
     longlat_dict[code] = [lt,ln]

#### Create columns longitude and latitude from geospatial data frame 

In [ ]:
lat=[]
long=[]
for code in result['Postcode']:
    lonlat = longlat_dict[code]
    lat.append(lonlat[0])
    long.append(lonlat[1])

#### Add the location details to result data frame

In [ ]:
result['Latitude'] = lat
result['Longitude'] = long
result

#### Get the longitude and latitude values of Toronto, Ontario

In [ ]:
address = 'Toronto, On'
geolocator =  Nominatim(user_agent="Toronto_Explorer")
location = geolocator.geocode(address)
longitude = location.longitude
latitude = location.latitude
print(latitude, longitude)

#### Create the toronto neighborhood map for analysis

In [ ]:
map_toronto = folium.Map(location =[latitude, longitude], zoom_start=12)

for borough, neighbourhood, lat, lng in zip(result['Borough'], result['Neighbourhood'], result['Latitude'], result['Longitude']):
    label = '{}, {}'.format(borough, neighbourhood)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup = folium.Popup(label, parse_html=True), 
        color='blue',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.7,      
        parse_html=False).add_to(map_toronto) 
   
    #label = folium.Popup(label,max_width=450, parse_html=True)
map_toronto

#### Four suqare credentials 

In [ ]:
CLIENT_ID = 'HQFIE2LGCMTESBTMPHN23QEPWMUYLRZDZ3AMZD4FWF5Z3QJA' # your Foursquare ID
CLIENT_SECRET = 'FAYQ2B50PC42VERLNA1MDG0HWDKMKRPC4ZAVJREEBDD13HEM' # your Foursquare Secret
VERSION = '20191204' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Exploring borough which we are interested

In [ ]:
borough_to_explore = 'Downtown Toronto'
dtown_toronto = result[result['Borough'] == borough_to_explore].reset_index(drop=True)
dtown_toronto

In [ ]:
address = borough_to_explore + ',Toronto' 

geolocator = Nominatim(user_agent = "dtown_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

#### Neighbourhood map of selected borough

In [ ]:
map_toronto = folium.Map(location =[latitude, longitude], zoom_start=12)

for borough, neighbourhood, lat, lng in zip(dtown_toronto['Borough'], dtown_toronto['Neighbourhood'],
                                            dtown_toronto['Latitude'], dtown_toronto['Longitude']):
    label = '{}, {}'.format(borough, neighbourhood)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup = folium.Popup(label, parse_html=True), 
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,      
        parse_html=False).add_to(map_toronto) 
   
    #label = folium.Popup(label,max_width=450, parse_html=True)
map_toronto

#### Create URL for searching venues

In [ ]:
radius = 500
limit = 100

url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(
        CLIENT_ID, CLIENT_SECRET, latitude, longitude,VERSION, radius, limit)
url

#### Get the response from the URL 

In [ ]:
response = requests.get(url).json()
response['response']['venues']
nearby_venues = json_normalize(response['response']['venues'])

#### Extract the categories from the venue

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

#### Function to get the near by venues of the neighbourhoods using foursquare API

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        results
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
dtown_toronto_venues = getNearbyVenues(names=dtown_toronto['Neighbourhood'],
                                   latitudes=dtown_toronto['Latitude'],
                                   longitudes=dtown_toronto['Longitude']
                                  )

In [ ]:
dtown_toronto_venues.head()

In [ ]:
dtown_toronto_venues.groupby('Neighborhood').count()

#### Add the venue categories as features using dummies

In [ ]:
dtown_toronto_onehot = pd.get_dummies(dtown_toronto_venues['Venue Category'])
dtown_toronto_onehot['Neighbourhood'] = dtown_toronto_venues['Neighborhood']

In [ ]:
fixed_columns = [dtown_toronto_onehot.columns[-1]] + list(dtown_toronto_onehot.columns[:-1])
dtown_toronto_onehot = dtown_toronto_onehot[fixed_columns]
dtown_toronto_onehot.head()

#### Group the neighbourhoods by mean for calculation

In [ ]:
dtown_toronto_grouped = dtown_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
dtown_toronto_grouped.dtypes
#for ind in np.arange(dtown_toronto_grouped.shape[0]):
#     print(dtown_toronto_grouped.iloc[ind])
      

In [ ]:
#dtown_toronto_grouped = dtown_toronto_grouped.iloc[:,1:]
#dtown_toronto_grouped 

#### Get the top 'k' venues of the neighbourhood 

In [ ]:
num_of_topvenues =10
for venue in dtown_toronto_grouped['Neighbourhood']:
    temp = dtown_toronto_grouped[dtown_toronto_grouped['Neighbourhood'] == venue].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
   # print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_of_topvenues))

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_of_topvenues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = dtown_toronto_grouped['Neighbourhood']

for ind in np.arange( dtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtown_toronto_grouped.iloc[ind, :], num_of_topvenues)
    
neighborhoods_venues_sorted.head()

#### Cluster the neighbourhoods

In [ ]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

dtown_toronto_grouped_clustering = dtown_toronto_grouped.drop('Neighbourhood', 1)
dtown_toronto_grouped_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

#### Insert cluster label into the result

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtown_toronto_merged = dtown_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dtown_toronto_merged = dtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dtown_toronto_merged.head() # check the last columns!

In [ ]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtown_toronto_merged['Latitude'], dtown_toronto_merged['Longitude'], dtown_toronto_merged['Neighbourhood'], dtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=1,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters


#### Cluster 1

In [ ]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 0, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

#### Cluster 2

In [ ]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 1, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

#### Cluster 3

In [ ]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 2, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

#### Cluster 4

In [ ]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 3, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]

#### Cluster 5

In [ ]:
dtown_toronto_merged.loc[dtown_toronto_merged['Cluster Labels'] == 4, dtown_toronto_merged.columns[[1,2] + list(range(5, dtown_toronto_merged.shape[1]))]]